In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from numba import jit, autojit
from scipy.sparse import csc_matrix, dia_matrix, diags
from scipy.sparse.linalg import eigs, eigsh
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor as NN
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import pandas as pd

x = np.linspace(-10,10,1001)

In [2]:
# Initialise constants for the softcore potential:
v = 200# set =0 for particle in a box case
n = 2
b = 2


@np.vectorize
def potential_softcore(xk, t):
    numerator = -v
    denominator = (np.abs(xk)**n + b**n)**(1/n)
    return numerator/denominator

@np.vectorize
def potential_linear_with_time(xk, t):
    alpha = 5
    numerator = -v
    denominator = (np.abs(xk)**n + b**n)**(1/n)
    return numerator/denominator + alpha*t*xk

@np.vectorize
def potential_oscillating_with_time(xk,t):
    omega = 5
    numerator = -v
    denominator = (np.abs(xk)**n + b**n)**(1/n)
    return numerator/denominator + np.sin(t*omega)*xk

"""
Choose which potential function to use:
"""
potential = potential_softcore

def gen_diag_Hamiltonian(x_arr):
    
    dx2 = -1/(2*(np.abs(x_arr[0]-x_arr[1])**2))
    
    centre_diag = -(5/2)*np.ones_like(x_arr)*dx2
    one_off_diag = (4/3)*np.ones_like(x_arr[:-1])*dx2    
    two_off_diag = -(1/12)*np.ones_like(x_arr[:-2])*dx2
    
    H = diags([centre_diag,one_off_diag,one_off_diag,two_off_diag,two_off_diag],[0,1,-1,2,-2])
    return H

def gen_diag_V(x_arr, potential_func, t):
    V = potential_func(x_arr, t)
    return diags([V],[0])

H_without_V = gen_diag_Hamiltonian(x) 

In [3]:
def describe_potential(V):
    """
    This returns information on the shape of the potential distribution,
    to be used by the eigenvalue-predictor
    """
    maxV = np.max(V)
    minV = np.min(V)
    varianceV = np.var(V)
    meanV = np.mean(V)
    medianV = np.median(V)
    return [maxV,minV,varianceV,meanV,medianV]

In [4]:

#Generate data for the random forrest model to train on

results = []
for v_val in np.linspace(0,1000,1001):
    #global v, results
    v = v_val
    pot_soft = potential_softcore(x, 0)
    H = H_without_V + gen_diag_V(x, potential_softcore, 0)
    min_eigval = eigsh(H, k=1, sigma=-550)[0][0]
    result = describe_potential(pot_soft)
    result.append(min_eigval)
    results.append(result)

df = pd.DataFrame.from_records(results,columns=['max','min','variance','mean','median','min_eigval'])
df.to_csv('PotentialsCorrespondingEigenvalues.csv')
df.tail()

,max,min,variance,mean,median,min_eigval
996,-97.665835,-498.0,15077.510078,-230.186307,-184.952557,-492.490153
997,-97.763893,-498.5,15107.801402,-230.417418,-185.138252,-492.987353
998,-97.861951,-499.0,15138.123123,-230.648529,-185.323948,-493.484555
999,-97.960010,-499.5,15168.475242,-230.879639,-185.509643,-493.981759
1000,-98.058068,-500.0,15198.857759,-231.110750,-185.695338,-494.478964


In [5]:
df.describe()

,max,min,variance,mean,median,min_eigval
count,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000
mean,-49.029034,-250.000000,5068.819063,-115.555375,-92.847669,-246.342235
std,28.349383,144.554056,4538.503659,66.815992,53.686028,143.261569
min,-98.058068,-500.000000,0.000000,-231.110750,-185.695338,-494.478964
25%,-73.543551,-375.000000,949.928610,-173.333063,-139.271504,-370.227726
50%,-49.029034,-250.000000,3799.714440,-115.555375,-92.847669,-246.115813
75%,-24.514517,-125.000000,8549.357489,-57.777688,-46.423835,-122.272922
max,-0.000000,-0.000000,15198.857759,0.000000,0.000000,0.012292


In [6]:
train, test = train_test_split(df, train_size=0.85)

#model = AdaBoostRegressor(loss='square', n_estimators=400, learning_rate=1) 
model = NN(hidden_layer_sizes=(100,90,100), solver='lbfgs')
train_inputs, train_outputs = train[['max','min','variance','mean','median']], train[['min_eigval']]
test_inputs, test_outputs = test[['max','min','variance','mean','median']], test[['min_eigval']]

/home/luke/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
model.fit(train_inputs, train_outputs)

/home/luke/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 90, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [8]:
model.score(test_inputs,test_outputs)

0.9999640342355152

In [9]:
print(model.predict(test.tail(1)[['max','min','variance','mean','median']]))
test.tail(1).min_eigval

[-434.2897954]


879   -434.327993
Name: min_eigval, dtype: float64

In [10]:
joblib.dump(model,'min_eigvalue_NN.joblib') # save the model for future use

['min_eigvalue_NN.joblib']

In [27]:
import timeit

In [28]:
def make_df_from_describe_potential(desc):
    return pd.DataFrame.from_records(desc,columns=['max','min','variance','mean','median'])

x_new = np.linspace(-10,10,10000)
v = np.random.randint(0,1000)
H_test_without_V = gen_diag_Hamiltonian(x_new) 
H_test = H_test_without_V + gen_diag_V(x_new, potential_softcore, 0)

In [138]:
%timeit eigsh(H_test, k=1, sigma=-10000000) 
# overestimating the most -ve eigenvalue in the guess, would be unfeasibly long to work out all eigenvals and sort

40.2 s ± 6.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit eigsh(H_test, k=1, sigma=model.predict(make_df_from_describe_potential([describe_potential(potential_softcore(x_new,0))]))[0])
# Made sure this timing includes the model predicting the eigenvalue, and all extra steps for a fair comparison

107 ms ± 6.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
